# LUNA Train Unet

# Dependency Descriptions
1. **keras**: is a high-level neural networks library (that allows for easy and fast prototyping)

In [1]:
from __future__ import print_function

import numpy as np
from keras.models import Model
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K

Using TensorFlow backend.


In [2]:
WORKING_PATH = "../../../../output/build-simple-model/"
IMG_ROWS = 512
IMG_COLS = 512

K.set_image_dim_ordering('th')  # Theano dimension ordering in this code
# dimension ordering is simply the order dimensions come in (ex: width, height, z)
# and this is using theano's ordering convention

**[Dice Coefficient Loss Function](https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient)**: compares the predicted and actual node mask (similar metric to what was used in Ultrasound Nerve Segmentation challenge that the U-net was originally written for)


Everything should be working as they did it in their tutorial first, so you are sure you understand their code (and how their code works). Then you can slowly change the code to fit your own ideas, so you are sure errors are not due to an error in the copying of the tutorial's code. 

Therefore training and predicting will be done on the typical train/test split (that the tutorial recommends) and after getting the tutorial to work successfully you can use 10 fold cross validation in place of it to choose a model, then train the model on the entire dataset and predict.

# Understanding Of Sequential Order of Code
## Loading / Preprocessing Training Data
```python
imgs_train = np.load(working_path+"trainImages.npy").astype(np.float32)
imgs_mask_train = np.load(working_path+"trainMasks.npy").astype(np.float32)

imgs_test = np.load(working_path+"testImages.npy").astype(np.float32)
imgs_mask_test_true = np.load(working_path+"testMasks.npy").astype(np.float32)
    
mean = np.mean(imgs_train)  # mean for data centering
std = np.std(imgs_train)  # std for data normalization

imgs_train -= mean  # images should already be standardized, but just in case
imgs_train /= std
```

## Actually Creating the Unet
*goal: to understanding exactly how this set of code*
### Steps:
1. Create the intial structure of a Unet
2. Create checkpoints for the unet to save its best weights (at that time period)
3. Give the unet an initial set of weights (optional)
3. Train the unet on trianing data (consisting of lung image, and node mask)

### Getting the Unet
```python
# where the return of the function should give you the "model"
model = get_unet()
```

### Creating the Unet 
*using keras define the initial structure of the model (layers, nodes, etc...)*

**so how does this code create the structure of a model?**

```python
def get_unet():
    inputs = Input((1,IMG_ROWS, IMG_COLS)) 
    conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(inputs)
    conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(pool1)
    conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(pool2)
    conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(pool3)
    conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(pool4)
    conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(conv5)

    up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=1)
    conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(up6)
    conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv6)

    up7 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=1)
    conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(up7)
    conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv7)

    up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=1)
    conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(up8)
    conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv8)

    up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=1)
    conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(up9)
    conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv9)

    conv10 = Convolution2D(1, 1, 1, activation='sigmoid')(conv9)

    model = Model(input=inputs, output=conv10)

    model.compile(optimizer=Adam(lr=1.0e-5), loss=dice_coef_loss, metrics=[dice_coef])

    return model
```

### Research into Unet
#### Sequential Models ([reference](https://keras.io/getting-started/sequential-model-guide/))
- Sequential Model: linear stack of layers
- tell model what input shape to expect (first layer must recieve info about input shape)
- before training a model, configure the learning process, which requires a `compile` method which contains:
  1. an optimizer: from existing optimizers, or instance of optimizer class, [reference](https://keras.io/optimizers/)
  2. a loss function: the object the model will try to minimize, existing loss function or just an objective function, [reference](https://keras.io/objectives/)
     - note custom objective functions have specific structures (like must have y_true, y_pred and return a scalar)
  3. list of metrics: existing metrics, custom metrics must return single tensor value, [reference](https://keras.io/metrics/)
- keras models: trained on Numpy arrays of input data and labels, use the `fit` function (sequential model api [complete reference](https://keras.io/models/sequential/))
##### Examples 
*[complete examples folder](https://github.com/fchollet/keras/blob/master/examples/cifar10_cnn.py)*
- good demonstration of CNN ([here](https://github.com/fchollet/keras/blob/master/examples/cifar10_cnn.py))

### Save the trained model at checkpoints
```python
model_checkpoint = ModelCheckpoint('unet.hdf5', monitor='loss', save_best_only=True)
```

### Use weights given by tutorial
```python
if use_existing:
    model.load_weights('./unet.hdf5')
```
### Train model on training data
```python
model.fit(imgs_train, imgs_mask_train, batch_size=2, nb_epoch=20, verbose=1, shuffle=True,
              callbacks=[model_checkpoint])
```
_**The final weights are what you want, with those weights you put them on the model and can start making predictions**_